<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/Q%26A_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 인공지능 PDF Q&A 챗봇 프로젝트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-dotenv
# 일단 종호의 openai API 키를 .env 파일에 넣어 놓음 => 나중에 바꾸기

In [ ]:
!pip install gradio==4.10.0

In [3]:
!pip install langchain_openai==0.3.7

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.4 MB/s eta 0:00:00


In [4]:
!pip install langchain-huggingface==0.1.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [5]:
!pip install langchain_community==0.3.18

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [6]:
!pip install faiss-cpu==1.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 20.1 MB/s eta 0:00:00


In [7]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough

# 환경 변수 불러오기(openai API 키)
load_dotenv('/content/drive/MyDrive/Colab Notebooks/.env')

# LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini")

# 텍스트 분리
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

# 임베딩 모델
hf_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 프롬프트 템플릿
message = """
당신은 사용자의 질문에 답변을 하는 친절한 AI 어시스턴트입니다.
당신의 입무는 주어진 문맥을 토대로 사용자 질문에 답하는 것입니다.
만약, 문맥에서 답변을 위한 정보를 찾을 수 없다면 '주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다' 라고 답하세요.
정보를 찾을 수 있다면 한글로 답변해 주세요.

## 주어진 문맥:
{context}

## 사용자 질문:
{input}
"""
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("human", message)
    ]
)

# 출력 파서
parser = StrOutputParser()

# 전역 변수
db = None
retriever = None
rag_chain = None

def load_pdf(file):
  global db, retriever, rag_chain

  # 저장된 파일로 로드
  loader = PyPDFLoader(file.name)
  docs = loader.load_and_split(text_splitter=text_splitter)

  db = FAISS.from_documents(docs, hf_embeddings)
  retriever = db.as_retriever(search_kwargs={"k": 3})

  rag_chain = {
      "context": retriever,
      "input": RunnablePassthrough()
  } | prompt_template | llm | parser

  return "PDF 파일이 성공적으로 업로드 및 처리되었습니다! 질문을 입력하세요."

def answer_question(question):
  if rag_chain is None:
    return "먼저 PDF 파일을 업로드하세요!"
  return rag_chain.invoke(question)

with gr.Blocks(theme=gr.themes.Soft()) as demo:
  gr.Markdown("""
  # 📄 인공지능 PDF Q&A 챗봇
  **PDF 파일을 업로드하고 질문을 입력하면 AI가 답변을 제공합니다!**
""")

  with gr.Row():
    with gr.Column(scale=1):
      file_input = gr.File(label="PDF 파일 업로드")
      upload_button = gr.Button("📤 업로드 및 처리")

    with gr.Column(scale=2):
      status_output = gr.Textbox(label="📢 상태 메시지")
      question_input = gr.Textbox(label="❓ 질문 입력", placeholder="궁금한 내용을 적어주세요.")
      submit_button = gr.Button("🤖 답변 받기")
      answer_output = gr.Textbox(label="📝 AI 답변")

  upload_button.click(load_pdf, inputs=file_input, outputs=status_output)
  submit_button.click(answer_question, inputs=question_input, outputs=answer_output)

demo.launch() # 웹 어플리케이션 실행

In [2]:
#!pip uninstall numpy -y
#!pip install --no-cache-dir numpy==1.26.4
# colab에 numpy 2.버전이 설치되어 있어서 버전 충돌남

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 153.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [1]:
import numpy
print(numpy.__version__)

1.26.4


In [ ]:
# Colab에 임베딩 모델 다운로드
#from huggingface_hub import snapshot_download
#snapshot_download(repo_id="BAAI/bge-m3", local_dir="/content/drive/MyDrive/EmbeddingModel/bge-m3")